# imports

In [7]:
import pandas as pd
import numpy as np


# Database connection

In [ ]:
# todo once the DB is up

# Get data

In [ ]:
positions_query = """
SELECT pp.PortfolioId, s.ISIN, pp.PortfolioCurreny, pp.MarketValueNet_CB
FROM [dbo].[PortfolioPosition] pp 
LEFT JOIN [dbo].[Security] s on pp.SecurityId=s.SecurityId
WHERE s.IsCash = 0 
"""

In [ ]:
# start_of_trading_window = ""
# transaction_query = """
# SELECT t.PortfolioId, t.ISIN
# FROM [dbo].[Transaction] t
# WHERE t.TransTradeDate >= '""" + start_of_trading_window + """'
# """

In [6]:
# todo: change this into queries once the DB is up
portfolio_ids = []
portfolio_isins = []
portfolio_currencies = []
market_values = []
positions = pd.DataFrame({'PortfolioId': portfolio_ids, 'ISIN': portfolio_isins, 'PortfolioCurreny': portfolio_currencies, 'MarketValueNet_CB': market_values})
# transaction_ids = []
# transaction_isins = []
# transactions = pd.DataFrame({'PortfolioId': transaction_ids, 'ISIN': transaction_isins})

# preprocess data

In [ ]:
# drop nan
positions.dropna()
# transactions.dropna()
# drop duplicates
positions.drop_duplicates()
# transactions.drop_duplicates()
# remove portfolios with too few (less than 5) transactions
counts = positions['PortfolioId'].value_counts()
filtered_indices = counts[counts >= 5].index.tolist()
positions = positions[positions['PortfolioId'].isin(filtered_indices)]
# remove investments owned by too few (less than 5) portfolios
counts = positions['ISIN'].value_counts()
filtered_indices = counts[counts >= 5].index.tolist()
positions = positions[positions['ISIN'].isin(filtered_indices)]

In [ ]:
# build portfolio - investment pairs
portfolio_list = positions["PortfolioId"].unique()
isin_list = positions["ISIN"].unique()

user_indices = []
item_indices = []
for index, row in positions.iterrows():
    user_indices += [portfolio_list.tolist().index(row["PortfolioId"])]
    item_indices += [portfolio_list.tolist().index(row["ISIN"])]

ratings = [1] * len(user_indices)

user_item_rating_df = pd.DataFrame({"User": user_indices,
                                    "Item": item_indices,
                                    "Rating": ratings})

# train/test split

In [ ]:
X = user_item_rating_df[["User", "Item"]].values
y = user_item_rating_df["Rating"].values

In [ ]:

X_train, X_test = X[0:(len(user_item_rating_df) * 0.7)], X[(len(user_item_rating_df) * 0.7):]
y_train, y_test = y[0:(len(user_item_rating_df) * 0.7)], y[(len(user_item_rating_df) * 0.7):]

# implement feasibility filters

# build model

In [ ]:
from scipy import sparse
from sklearn.decomposition import nmf

In [ ]:
# get sparse representation
X_sparse = sparse.csr_matrix((y_train, (X_train[:, 0], X_train[:, 1])),
                             shape=(len(portfolio_list),len(isin_list)))

In [ ]:
W, H, _ = nmf.non_negative_factorization(X=X_sparse,
                                         W=None,
                                         H=None,
                                         n_components=5,
                                         init='random',
                                         update_H=True,
                                         solver='cd',
                                         beta_loss='frobenius',
                                         max_iter=200,
                                         tol=0.0001,
                                         alpha=0,
                                         l1_ratio=0,
                                         regularization='both',
                                         random_state=0,
                                         verbose=0,
                                         shuffle=False)

# predict

In [ ]:
# add some zero values to the test set ["User", "Item"]
length = X_test * 2
while len(X_test) < length:
    random_user_index = random.randint(0, len(portfolio_list))
    random_item_index = random.randint(0, len(isin_list))
    already_in = False
    for index, row in X_test.iterrows():
        if row['User'] == random_user_index and row['Item'] == random_item_index:
            already_in = True
    if not already_in:
        X_test = np.append(X_test, [[random_user_index, random_item_index]], axis=0)
        y_test = np.append(y_test, 0)

In [ ]:
y_pred = []
for index in X_test:
    # compute dot product
    dot_product = W[index[0], :].dot(H[:, index[1]])
    y_pred.append(dot_product)

# visualize metrics

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0

index = 0
for prediction in y_pred:
    true_value = y_test[index]
    if prediction == 0 and true_value == 0:
        TN += 1
    if prediction == 0 and true_value == 1:
        FN += 1
    if prediction == 1 and true_value == 0:
        FP += 1
    if prediction == 1 and true_value == 1:
        TP += 1
    index += 1

print("Precision:", TP/(TP+FP))
print("Recall:", TP/(TP+FN))